In [4]:
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox
#from tkinter.filedialog import *
import tkinter
from tkinter import ttk
import pandas as pd
import json
from docxtpl import DocxTemplate
import os
import datetime as dt
import locale
locale.setlocale(locale.LC_TIME,'')

'French_France.1252'

### Algo de génération des fichiers word

In [5]:
def word_generate(doc_folder, excel_folder, file_field, dest_folder, col_file_name, var1=None, var2=None):
    # doc_folder correspond au répertoire contenant les fichiers word à alimenter
    # file_field correspond à la colonne contenant les noms des fichiers à exporter
    # dest_folder correspond au répertoire de destination des fichiers à exporter
    # col_file_name correspond à la liste des colonnes utilisées pour nommer les fichiers    
    # Etape 1 chargement du fichier excel alimentant les fichiers word
    # var1 Nom du champ exploité pour ranger les résultats, création de répertoire
    # var2 Nom du champ exploité pour ranger les résultats, création de répertoire
    print("yes!!!")
    print(var1)
    # Etape 1 chargement du fichier excel contenant les valeurs destinées à alimenter les templates word
    augm=pd.read_excel(excel_folder)    
    list_context = augm.columns
    
    # Etape 2, gestion des dates dans le fichier excel
    for col in augm.columns:
        if str(augm[col].dtypes)=="datetime64[ns]":
            augm[col]= augm[col].dt.date
            for i, val in enumerate(augm.iterrows()):
                date_str=str(augm.at[i,col])
                if str(augm.at[i, col]) != "NaT":
                    augm.at[i, col] = dt.datetime.strptime(date_str, '%Y-%m-%d').date()
                    augm.at[i, col] = augm.at[i, col].strftime("%d %B %Y")
            augm[col] = augm[col].astype(str)
    
    # Etape 3 création du contexte
    for i, val in enumerate(augm.iterrows()):
        context={}
        for col in list_context:
            context[col] = augm.at[i, col]
        try:
            folder_world = doc_folder + "\\" + augm.at[i, file_field]
        except:
            print("fichier introuvable")
         
    # Etape 4 Création du nom d'export
        name_file = ""
        for col in col_file_name:
            if name_file =="":
                name_file= augm.at[i,col]
            else:
                name_file = name_file + " " + augm.at[i,col]
        name_file = name_file + ".docx"
    
    
    # Etape 5 Gestion des répertoires d'export dans le cas d'une ventillation des exports suivant des champs
        if var1 =="" and var2=="":   # Option 1: pas de variable
            dest = dest_folder + r"\\" + name_file

        if var1 != "" and var2 != "": # Option 3: 2 variables
            dest1 = dest_folder + "\\" + augm.at[i,var1]
            dest2 = dest1 + "\\" + augm.at[i,var2]
            if os.path.exists(dest2)== False:
                try:
                    os.mkdir(dest1)
                    os.mkdir(dest2)                
                except OSError:
                    print("Creation of the directory %s failed" % dest)
            dest = dest2 + "\\" + name_file

        if var1 != "" and var2 == "": # Option 2: 1 variable
            dest = dest_folder + r"\\" + augm.at[i,var1] + "\\" 
            if os.path.exists(dest)== False:
                try:
                    os.mkdir(dest) 
                except:
                    print("Creation of the directory %s failed" % dest)
            dest= dest + r"\\" + name_file
        # Etape 6: Génération des words
        doc= DocxTemplate(folder_world)
        doc.render(context)
        doc.save(dest)    

In [6]:
from tkinter import filedialog
from tkinter import *
from tkinter import messagebox
#from tkinter.filedialog import *
import tkinter
from tkinter import ttk
import pandas as pd
import json


class Application(Tk):


    def __init__(self, parent):

        Tk.__init__(self, parent)

        self.parent = parent
        self.columns=[]

        self.initialize()

        self.title("Publipostage")
        
        self.resizable(width=False, height=False)

        self.load_data()

        self.mainloop()


    def initialize(self):

        self.control_menu = Frame(self)

        self.control_menu.pack()


        self.conf_menu = LabelFrame(self.control_menu, text="Configuation", width=430, height=300, labelanchor='n')

        self.conf_menu.grid(row=1, column=1, pady=10, padx=10)


        self.rep_source_affiche = StringVar()
        self.rep_word_affiche = StringVar()

        self.rep_cible_affiche = StringVar()

        # Selection fichier source
        self.entree_label = Label(self.conf_menu, text="Référentiel excel source")

        self.entree_label.grid(row=1, column=1, padx=20,sticky=W)


        self.entree = Entry(self.conf_menu, textvariable=self.rep_source_affiche, width=50)

        self.entree.grid(row=1, column=2,sticky=W, padx=0)


        self.imp_Button = Button(self.conf_menu, text="Sélection...", command=self.import_file)

        self.imp_Button.grid(row=1, column=3, padx=15, pady=10,sticky=W)
        
        #Selection repertoire word
        
        self.word_label = Label(self.conf_menu, text="Répertoire fichiers word")

        self.word_label.grid(row=2,column=1, padx=20,sticky=W)
        

        self.word_entree = Entry(self.conf_menu, textvariable=self.rep_word_affiche, width=50)

        self.word_entree.grid(row=2, column=2, sticky=W, padx=0)


        self.word_Button = Button(self.conf_menu, text="Sélection...", command = self.word_cible)

        self.word_Button.grid(row=2, column=3, padx=15, pady=10,sticky=W)
        
        
        # Selection répertoire cible
        
        self.cible_label = Label(self.conf_menu, text="Répertoire cible")

        self.cible_label.grid(row=3,column=1, padx=20,sticky=W)


        self.cible_entree = Entry(self.conf_menu, textvariable=self.rep_cible_affiche, width=50)

        self.cible_entree.grid(row=3, column=2, sticky=W, padx=0)


        self.cible_Button = Button(self.conf_menu, text="Sélection...", command = self.rep_cible)

        self.cible_Button.grid(row=3, column=3, padx=15, pady=10,sticky=W)
        
        # Selection critère de validation 1
        
        self.crit1_label = Label(self.conf_menu, text="Critère de ventillation 1")
        self.crit1_label.grid(row=4,column=1, padx=20,sticky=W)
        
        self.list_var1 = ttk.Combobox(self.conf_menu, values=self.columns)
        self.list_var1.grid(row=4,column=2,pady=10, sticky=W)
        
        # Selection critère de validation 2
        
        self.crit2_label = Label(self.conf_menu, text="Critère de ventillation 2")
        self.crit2_label.grid(row=5,column=1, padx=20,sticky=W)
        
        self.list_var2 = ttk.Combobox(self.conf_menu, values=self.columns)
        self.list_var2.grid(row=5,column=2,pady=10,sticky=W)        
        
        
        # Menu Execution

        self.ex_menu = LabelFrame(self.control_menu, text="Execution", width=430, height=200, labelanchor='n')

        self.ex_menu.grid(row=1, column=2,padx=10, pady=10)


        self.ex_Button = Button(self.ex_menu, text="Executer", command = self.ex_programme,height = 1, width = 18)

        self.ex_Button.grid(row=1, column=1, padx=10, pady=10,sticky=W)


        self.quitButton = Button(self.ex_menu, text="Quitter", command=self.destroy,height = 1, width = 18)

        self.quitButton.grid(row=2, column=1, padx=10, pady=10,sticky=W)


    def import_file(self):

        rep = filedialog.askopenfilename(initialdir="/", title="Select file",

                                         filetypes=(("Excel files", "*.xlsx"), ("all files", "*.*")))

        if rep != "":

            self.rep_source_affiche.set(rep)

            self.source = pd.read_excel(rep)
            
            self.columns = list(self.source.columns)
            self.list_var1.config(values=self.columns) # alimentation de var1
            self.list_var2.config(values=self.columns) # alimentation de var2

            # Sauvegarde du réperoire source
            try:

                self.save_data["source"] = rep

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)

            except:

                self.save_data.update({"Source": rep})

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)
                    
            # Sauvegarde list var
            
            try:
                self.save_data["Liste_var"] = self.columns

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)

            except:

                self.save_data.update({"Liste_var": self.columns})

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)
                     
            
    def rep_cible(self):


        self.rep_cible = filedialog.askdirectory()

        if self.rep_cible != "":

            self.rep_cible_affiche.set(self.rep_cible)


            try:

                self.save_data["cible"] = self.rep_cible

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)

            except:

                self.save_data.update({"Cible": self.rep_cible })

                with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)
                    
    def word_cible(self):
        
        self.rep_word = filedialog.askdirectory()
        if self.rep_word != "":
            self.rep_word_affiche.set(self.rep_word)
        
        try:
            self.save_data["Cible_word"] = self.rep_word
            
            with open('data.txt', 'w+') as outfile:

                json.dump(self.save_data, outfile)
        except:
            
            self.save_data.update({"Cible_word": self.rep_word })

            with open('data.txt', 'w+') as outfile:

                    json.dump(self.save_data, outfile)
            

    def ex_programme(self):

        if self.rep_source_affiche.get() == "":

            messagebox.showerror("Ficher source non sélectionné", "Veuillez sélectionner un fichier source")


        if self.rep_cible_affiche.get() =="":

            messagebox.showerror("Répertoire source non sélectionné", "Veuillez sélectionner un répertoire source")

        if self.rep_source_affiche.get() != "" and self.rep_cible_affiche.get() != "":
            print(self.rep_word_affiche.get())
            print(self.rep_source_affiche.get())
            print(self.rep_cible_affiche.get())
            print(self.list_var1.get())
            print(self.list_var2.get())
            
            #word_generate(self.rep_word_affiche.get(), self.rep_source_affiche.get(), "Fichiers_export",["Nom","Prénom","MYHR_ID"],self.rep_cible_affiche.get(),self.list_var1.get(),self.list_var2.get())
            word_generate(doc_folder=self.rep_word_affiche.get(),excel_folder=self.rep_source_affiche.get(), file_field="Fichiers_export",col_file_name=["Nom","Prénom","MYHR_ID"], dest_folder=self.rep_cible_affiche.get(),var1= self.list_var1.get(),var2 = self.list_var2.get())  

    def load_data(self):

        try:

            with open('data.txt') as json_file:
                self.save_data = json.load(json_file)

            try:

                self.rep_cible_affiche.set(self.save_data["cible"])

            except:

                pass

            try:
                self.rep_source_affiche.set(self.save_data["source"])

            except:

                pass
            
            try:
                self.list_var1.config(values=self.save_data["Liste_var"])
                self.list_var2.config(values=self.save_data["Liste_var"])
            
            except:
                pass
            
            try:
                self.rep_word_affiche.set(self.save_data["Cible_word"])
            except:
                pass
        except:

            print("nope")

            self.save_data={}

toto = Application(None)

toto.mainloop()

C:/Users/sgasmi/Desktop/Dev/Com&Ben/Notification des Augmentations/Template
C:/Users/sgasmi/Desktop/Dev/Com&Ben/Notification des Augmentations/Bordereau pour Notifications_IBF fichier test.xlsx
C:/Users/sgasmi/Desktop/Dev/Com&Ben/Export
Civilité

yes!!!
Civilité
C:/Users/sgasmi/Desktop/Dev/Com&Ben/Notification des Augmentations/Template
C:/Users/sgasmi/Desktop/Dev/Com&Ben/Notification des Augmentations/Bordereau pour Notifications_IBF fichier test.xlsx
C:/Users/sgasmi/Desktop/Dev/Com&Ben/Export
Civilité

yes!!!
Civilité
